In [12]:
import matplotlib.pyplot as plt
import time 
import numpy as np
from astropy.io import fits
import os
directorio='/Users/oscar/Documents/Documentos/Docs/LME/CCD/OscuraImagenes/'
#directorio='/Users/oscar/Documents/Documentos/Docs/LME/CCD/ImagenPrueba/'
contenido = os.listdir(directorio)

In [13]:
# Loop sobre todas las imagenes, se guarda en img_total cada ccd en fila demultiplexado
st = time.time()
#data=fits.HDUList([])
img_CCD16=fits.HDUList([]) #Se crearan nro_imagenes*nsamp/16 (Ej: 5*112/6=35)

for item in range(0,len(contenido)):
    hdulist = fits.open(directorio+contenido[item])
    tamx=int(hdulist[4].header['NAXIS1']) #134400
    tamy=int(hdulist[4].header['NAXIS2']) #Varia 
    nsamp=int(hdulist[4].header['NSAMP']) #112
    ncol=int(hdulist[4].header['NCOL'])
    ccdncol=int(hdulist[4].header['CCDNCOL'])
    scidata = hdulist[4].data
    tamxpimg=int(tamx/nsamp)
    div=16 #Variable multiplo de 2^n 
    for j in range(int(nsamp/16)):                        #Se recorre nsamp/16=7 veces por cada imagen 
        img_parcial= np.zeros((tamy,int(div*(tamx/nsamp))),'i4')  #Se crean las imagenes parciales 
        for i in range(div):                               #Se recorre 16 veces para ir agarrando 16 CCD  
            for k in range(tamxpimg):              #Se recorre tamx/nsamp=1200 veces para agarrar un CCD
                img_parcial[:,k+i*tamxpimg]=scidata[:,(k*nsamp)+(i+div*j)] 
            for p in range(tamy):
                #print("Se sacara el offset de las columnas ",tamxpimg*(i+1)-int(ncol-ccdncol/2),"hasta la ", tamxpimg*(i+1), "en la fila ",p)
                offset=np.mean(img_parcial[p,(tamxpimg*(i+1)-int(ncol-ccdncol/2)):tamxpimg*(i+1)])
                #print("el offset es",offset)
                img_parcial[p,tamxpimg*(i):tamxpimg*(i+1)]=img_parcial[p,tamxpimg*(i):tamxpimg*(i+1)]-offset
        img_CCD16.append(fits.ImageHDU(img_parcial))        #Se agrega una imagen a la lista tras recorrer 16 CCD
    #Proceso de guardado
    nombre=str("Imagen"+str(item)+".fits")
    hdul=fits.HDUList(img_CCD16)    
    hdul.writeto(nombre,overwrite=True)
    print('Se termino la imagen ',item)
    img_CCD16.clear()
et = time.time()
elapsed_time = et - st
print('Execution time:', elapsed_time, 'seconds')

Se termino la imagen  0
Se termino la imagen  1
Se termino la imagen  2
Se termino la imagen  3
Se termino la imagen  4
Execution time: 27.37057638168335 seconds
